In [ ]:
import pandas as pd
from pathlib import Path
from itertools import product

In [ ]:
filename = Path('../databases/ouro_npk-balanco.parquet')
index_cols = ["fluxo", "fonte", "elemento"]

data = (
    pd.read_parquet(filename)
        .unstack(level=["ano"]).droplevel([0], axis="columns")
        .reset_index(level=index_cols)
        .join(
            pd.read_parquet("../databases/prata_geocodigo_ibge.parquet")
                .loc[:, ["UF-id", "UF-sigla"]]
                .drop_duplicates()
                .rename(columns={"UF-id": "id", "UF-sigla": "uf"})
                .set_index("id")
                .squeeze()
        )
        .set_index(["uf"] + index_cols)
)

data

In [ ]:
elements = (
    data.index
        .get_level_values("elemento")
        .dropna()
        .unique()
        .tolist()
)

ufs = (
    data.index
        .get_level_values("uf")
        .dropna()
        .unique()
        .tolist()
)

# Por UF
for element, uf in  product(elements, ufs):
    out_xlsx = Path(f"../databases/isagro_balanco_{element.lower()}.xlsx")

    if out_xlsx.exists():
        kwargs = {"mode": "a", "if_sheet_exists": "replace"}
    else:
        kwargs = {"mode": "w"}

    with pd.ExcelWriter(out_xlsx, engine="openpyxl", **kwargs) as writer:
        group = ["uf", "elemento"]
        print(element, uf)

        (
            data.groupby(group)
                .get_group((uf, element))
                .droplevel(group)
                .to_excel(writer, sheet_name=uf)
        )

# Nacional
for element in elements:
    out_xlsx = Path(f"../databases/isagro_balanco_{element.lower()}.xlsx")
    
    with pd.ExcelWriter(out_xlsx, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        (
            data.groupby(index_cols)
                .sum()
                .groupby("elemento")
                .get_group(element)
                .droplevel("elemento")
                .to_excel(writer, sheet_name="Brasil")
        )
